# 챗봇 엔진 만들기

## 들어가기 전에

**챗봇 엔진**이란 챗봇에서 핵심 기능을 하는 자연어 처리 모듈로, 화자의 질문을 이해하고 알맞은 답변을 출력하는 역할을 합니다.

챗봇 엔진을 설계하기 전에는 챗봇의 목적과 도메인을 결정해야 합니다. 이에 따라 챗봇 엔진 개발 방법론과 학습에 필요한 데이터셋이 달라집니다. 이 글의 예시에서는 음식 예약 및 주문을 도와주는 챗봇 엔진을 만들겠습니다.

챗봇 엔진의 처리 과정은 다음 네 단계로 나타낼 수 있습니다.

![챗봇 엔진의 처리 과정]

1. **전처리 과정**: 화자의 질의 문장이 입력되면 챗봇 엔진은 제일 먼저 전처리를 진행합니다. 형태소 분석기를 이용해 단어 토큰을 추출한 뒤 명사나 동사 등 문장 해석에 의미 있는 품사만 남기고 불용어는 제거합니다.
2. **질문 의도 분류**: 화자의 질문 의도를 파악합니다. 즉 의도 분류 모델을 이용해 의도 클래스를 예측합니다.
3. **개체명 인식**: 화자의 질문에서 단어 토큰별 개체명을 인식합니다.
4. **답변 검색**: 해당 질문의 의도, 개체명, 핵심 키워드를 기반으로 답변을 학습 DB에서 검색합니다.

앞으로의 글은 위 네 단계를 순차적으로 다룰 것입니다. 추가적으로 의도 분류 및 개체명 인식 모델의 학습을 하려면 단어 사전을 구축해야 하기 때문에 **단어 사전 구축 및 시퀀스 생성 방법**을 알아볼 것입니다. 또한 다양한 플랫폼에서 우리가 만든 챗봇 엔진에 언제든 접속할 수 있도록 **챗봇 엔진 서버 프로그램 개발**에 대한 내용도 살피도록 하겠습니다.

In [ ]:
pip install konlpy
pip install tensorflow
pip install seqeval
pip install pymysql

## 1. 전처리 과정

전처리를 담당하는 모듈을 만들어봅시다. /utils 디렉터리에 Preprocess.py라는 파일을 생성하도록 하겠습니다. 챗봇 엔진 내에서 자주 사용하기 때문에 클래스로 정의합니다.

`pos` 함수는 형태소 분석기로 토크나이징 작업을 담당하고, `get_keywords` 함수는 문장 해석에 의미 있는 정보만 남기고 나머지 불용어들은 제거하는 작업을 합니다. 마지막으로 `get_wordidx_sequence` 함수는 다음 목차에서 구축할 단어 사전을 이용해 입력된 문장을 단어 시퀀스 벡터로 변환합니다.

In [ ]:
from konlpy.tag import Komoran
import pickle

class Preprocess:
	# 생성자
	def __init__(self, word2index_dic="", userdic=None):
		# 단어 인덱스 사전 불러오기
		if word2index_dic != "":
			f = open(word2index_dic, "rb")
			self.word_index = pickle.load(f)
			f.close()
		else:
			self.word_index = None

		# 형태소 분석기 초기화
		self.komoran = Komoran(userdic=userdic)

		# 제외할 품사
		# 참조: https://docs.komoran.kr/firststep/postypes.html
		self.exclusion_tags = [
			"JKS", "JKC", "JKG", "JKO", "JKB", "JKV", "JKQ", "JX", "JC", # 관계언 제거
			"SF", "SP", "SS", "SE", "SO", # 기호 제거
			"EP", "EF", "EC", "ETN", "ETM", # 어미 제거
			"XSN", "XSV", "XSA", # 접미사 제거
		]

	# 형태소 분석기 POS tagger (래퍼 함수)
	def pos(self, sentence):
		return self.komoran.pos(sentence)

	# 불용어 제거 후 필요한 품사 정보만 가져오기
	def get_keywords(self, pos, without_tag=False):
		f = lambda x: x in self.exclusion_tags
		word_list = []
		for p in pos:
			if f(p[1]) is False: # 불용어 리스트에 없는 경우에만 저장
				word_list.append(p if without_tag is False else p[0])
		return word_list

	# 키워드를 단어 인덱스 시퀀스로 변환
	def get_wordidx_sequence(self, keywords):
		if self.word_index is None:
			return []
		w2i = []
		for word in keywords:
			try:
				w2i.append(self.word_index[word])
			except KeyError:
				# 해당 단어가 사전에 없는 경우 OOV 처리
				w2i.append(self.word_index["OOV"])
		return w2i

## 2. 단어 사전 구축 및 시퀀스 생성

/train_tools/dict 디렉터리에 있는 말뭉치 데이터 corpus.txt를 활용하여 단어 사전을 생성해봅시다. 같은 디렉터리에 create_dict.py 파일을 만들도록 하겠습니다. 이 파일에서 생성된 단어 사전은 의도 분류 및 개체명 인식 모델의 학습에 활용될 예정입니다.

In [ ]:
from utils.Preprocess import Preprocess
from tensorflow.keras import preprocessing
import pickle

# 말뭉치 데이터 읽어오기
def read_corpus_data(filename):
	with open(filename, "r") as f:
		data = [line.split("\t") for line in f.read().splitlines()]
		data = data[1:] # 헤더 제거
	return data

# 말뭉치 데이터 가져오기
corpus_data = read_corpus_data("./train_tools/dict/corpus.txt")

# 말뭉치 데이터에서 키워드만 추출해서 사전 리스트 생성
p = Preprocess()
dict = []
for c in corpus_data:
	pos = p.pos(c[1])
	for k in pos:
		dict.append(k[0])

# 사전에 사용될 단어 인덱스 딕셔너리(word_index) 생성
tokenizer = preprocessing.text.Tokenizer(oov_token="OOV")
tokenizer.fit_on_texts(dict)
word_index = tokenizer.word_index

# 사전 파일 생성
f = open("chatbot_dict.bin", "wb")
try:
	pickle.dump(word_index, f)
except Exception as e:
	print(e)
finally:
	f.close()

우리가 사용한 단어가 사전에 존재하지 않을 수 있습니다. 이 경우 챗봇 엔진에서는 OOV (Out-of-Vocabulary) 처리를 합니다. 위 코드에서 oov_token을 “OOV”라는 string으로 설정하면 사전의 첫 번째 인덱스에 “OOV”가 저장됩니다. 따라서 나중에 단어 인덱스 시퀀스를 만들 때 어떤 단어가 사전에 없는 경우 “OOV”의 인덱스를 저장하는 식으로 처리해주면 됩니다.

## 3. 의도 분류 모델

챗봇 엔진에 화자의 질의가 입력되었을 때, 전처리 과정을 거친 후 해당 문장의 의도를 분류합니다. 문장을 의도 클래스별로 분류하기 위해 이전에 사용했던 모델인 CNN을 사용하겠습니다. 다양한 의도를 분류하기에는 학습 데이터 수가 한정적이기 때문에 5가지 의도로만 분류할 수 있도록 구현하겠습니다.

- 챗봇 엔진의 의도 분류 클래스 종류

모델 학습 모듈을 만들기 전에 챗봇 엔진 소스 전역에서 사용할 파라미터 정보를 /config 디렉터리 내에 파일로 관리하겠습니다. [GlobalParams.py](http://GlobalParams.py) 파일을 생성해 다음과 같이 작성하세요.

In [ ]:
# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN =15

def GlobalParams():
	global MAX_SEQ_LEN

### 3.1 의도 분류 모델 학습

챗봇 엔진의 의도 분류 모듈을 만들기 전에 해당 모델의 설계 및 학습을 진행하겠습니다.

/model/intent 디렉터리에 total_train_data.csv 파일을 학습 데이터셋으로 사용합니다. 이 파일은 이전에 사용한 chatbot_data.csv(송영숙 님 공개 데이터)을 기반으로 생성한 의도 분류용 학습 데이터셋입니다. 이 데이터는 음식점 주문과 예약을 위한 챗봇에 특화되어 있으며, 클래스별 샘플 텍스트가 다양하지 않아 특정 의도 클래스인 경우 예측 품질이 떨어질 수 있습니다.

다음은 total_train_data.csv 파일을 읽어와 의도 분류 모델을 생성하고 학습하는 코드입니다.

해당위치(/models/intent 디렉터리)에 train_model.py 파일을 생성하세요.

- 챗봇 엔진 의도 분류 모델

total_train_data.csv 파일을 읽어와 CNN 모델 학습 시 필요한 query(질문)과 intent(의도)를 리스트에 저장합니다. 그 다음 챗봇 전처리 모듈 Preprocess로 단어 시퀀스를 생성합니다. 해당 단어에 매칭되는 번호로 시퀀스를 생성합니다.

In [ ]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate


# 데이터 읽어오기
train_file = "./models/intent/total_train_data.csv"
data = pd.read_csv(train_file, delimiter=',')
queries = data['query'].tolist()
intents = data['intent'].tolist()

from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dict.bin',
               userdic='./utils/user_dic.tsv')

# 단어 시퀀스 생성
sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

위에서 생성한 단어 시퀀스 벡터의 크기를 동일하게 맞춰주기 위해 MAX_SEQ_LEN 크기만큼 시퀀스 벡터를 패딩 처리합니다.

In [ ]:
# 단어 인덱스 시퀀스 벡터
# 단어 시퀀스 벡터 크기
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

패딩 처리된 시퀀스 벡터 리스트와 intent(의도) 리스트 전체를 데이터셋 객체로 만듭니다. 그 다음 데이터를 랜덤으로 섞고 학습용, 검증용, 테스트용 데이터 셋으로 7:2:1 비율로 나눠 실제 학습에 필요한 데이터셋 객체를 각각 분리합니다.

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

케라스 함수형 모델 방식으로 의도 분류 모델을 구현합니다. 입력하는 문장을 의도 클래스로 분류하는 CNN모델은 여러 영역으로 구성되어 있습니다. 첫 번째로 입력 데이터를 단어 임베딩 처리하는 영역, 그 다음으로 합성곱 필터와 연산을 통해  특징맵을 추출하고 평탄화하는 영역, 완전 연결 계층을 통해 감정별로 클래스를 분류하는 영역으로 구성되어 있습니다.

In [ ]:
# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수


# CNN 모델 정의  ○4
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

우리는 5가지 의도 클래스를 분류해야하기에 출력노드가 5개인 Dense 계층을 생성합니다. 마지막으로 출력 노드로 정의한 logits에서 나온 함수를 소프트맥스 계층을 통해 감정 클래스별 확률을 계산합니다.

In [ ]:
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(5, name='logits')(dropout_hidden)
predictions = Dense(5, activation=tf.nn.softmax)(logits)

위에서 정의한 계층들을 케라스 모델에 추가하는 작업을 진행합니다. 모델 정의 후 실제 모델을 model.compile() 함수를 통해 CNN모델을 컴파일합니다. 최적화 방법에는 adam, 손실함수에는 sparse_categorical_crossentropy, 모델 평가할 때 정확도 확인하기 위해 metrics에 accuracy를 사용하도록 했습니다.

In [ ]:
#모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

모델을 학습하기 위해 model.fit() 함수를 사용합니다. 에포크값을 5로 설정했으므로 모델 학습을 5회 반복합니다. 또한 evaluate() 함수를 이용해 성능을 평가합니다. 인자에는 테스트용 데이터셋을 사용합니다. 마지막으로 학습이 완료된 모델을 h5 파일 포맷으로 저장합니다. 해당 모델 파일은 챗봇 엔진의 의도 분류 모델에서 사용됩니다.

In [ ]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)


# 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))


# 모델 저장
model.save('intent_model.h5')

Epoch 1/5
3698/3698 [==============================] - 204s 54ms/step - loss: 0.0503 - accuracy: 0.9844 - val_loss: 0.0169 - val_accuracy: 0.9945
Epoch 2/5
3698/3698 [==============================] - 188s 51ms/step - loss: 0.0164 - accuracy: 0.9944 - val_loss: 0.0086 - val_accuracy: 0.9966
Epoch 3/5
3698/3698 [==============================] - 193s 52ms/step - loss: 0.0115 - accuracy: 0.9956 - val_loss: 0.0076 - val_accuracy: 0.9971
Epoch 4/5
3698/3698 [==============================] - 195s 53ms/step - loss: 0.0110 - accuracy: 0.9959 - val_loss: 0.0062 - val_accuracy: 0.9973
Epoch 5/5
529/529 [==============================] - 3s 5ms/step - loss: 0.0049 - accuracy: 0.9980
Accuracy: 99.801230
loss: 0.004867


### 3.2 의도 분류 모듈 생성

이제 챗봇 엔진의 의도 분류 모듈을 만들겠습니다. 이 모듈은 앞서 학습한 의도 분류 모델 파일을 활용해 입력되는 텍스트의 의도 클래스를 예측하는 기능을 가지고 있습니다. 해당 모듈은 딥러닝 모델이기에 /model/intent 디렉터리 내에 intentModel.py로 생성해주세요.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing


# 의도 분류 모델 모듈
class IntentModel:
    def __init__(self, model_name, proprocess):

        # 의도 클래스 별 레이블
        self.labels = {0: "인사", 1: "욕설", 2: "주문", 3: "예약", 4: "기타"}

        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 Preprocess 객체
        self.p = proprocess


    # 의도 클래스 예측
    def predict_class(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 단어 시퀀스 벡터 크기
        from config.GlobalParams import MAX_SEQ_LEN

        # 패딩처리
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

        predict = self.model.predict(padded_seqs)
        predict_class = tf.math.argmax(predict, axis=1)
        return predict_class.numpy()[0]

다음은 IntentModel 클래스를 테스트하는 코드입니다. IntentModel 객체를 생성해 새로운 유형의 문장을 분류합니다. 테스트 코드이므로 /test 디렉터리에 model_intent_test.py로 생성해주세요. predict는 query에 대한 예측 클래스 정보를 나타내고 predict_label은 query에 대한 예측 레이블을 나타냅니다. 학습 데이터에서 크게 벗어나지 않는 문장의 경우 의도를 적절하게 잘 예측합니다.

In [ ]:
from utils.Preprocess import Preprocess
from models.intent.IntentModel import IntentModel

p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dict.bin',
               userdic='../utils/user_dic.tsv')

intent = IntentModel(model_name='./models/intent/intent_model.h5', proprocess=p)
query = "오늘 탕수육 주문 가능한가요?"
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

1/1 [==============================] - 2s 2s/step


## 4. 개체명 인식 모델 학습 (영현)

다음으로 챗봇 엔진에 입력된 문장 의도 분류 후, 문장 내 개체명 인식(Named Entity Regocnition, NER) 모델을 만들어봅시다. 개체명 인식을 위해 지난 [3-1편에서 배운 양방향 LSTM](https://dacon.io/competitions/official/235946/codeshare/5812) 모델을 사용해보도록 하겠습니다.

개체명 인식 모델에서 인식 가능한 주요 개체명은 다음과 같습니다.

- **개체명 종류**

### 4.1 개체명 인식 모델 학습

/models/ner 디렉터리의 ner_train.txt 파일을 학습 데이터셋으로 사용합니다. 해당 파일은 지난 3편에서 사용한 train.txt을 기반으로 저자가 기존 학습 데이터셋에서 음식, 날짜, 시간 BIO 태그 데이터를 보강하여 생성한 개체명 인식용 학습 데이터셋입니다.

/models/ner 디렉터리 위치에서 train_model.py 파일을 생성하여 ner_train.txt 파일을 읽어와 NER 모델을 생성하고 학습하는 코드를 작성해보겠습니다.

(모델 학습 데이터 처리 부분만 다를 뿐 지난 3-1편에서의 개체명 인식 모델과 거의 동일한 소스 코드입니다.)

- **챗봇 엔진 NER 모델**

In [ ]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))

    return sents

# 전처리 객체 생성
p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dict.bin',
                  userdic = './utils/user_dic.tsv')

# 학습용 말뭉치 데이터 불러옴
corpus = read_file('./models/ner/ner_train.txt')

개체 인식 모델을 학습하기 위한 학습용 말뭉치 데이터를 read_file 함수를 통해 가져옵니다.

In [ ]:
# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
  tagged_sentence = []
  sentence, bio_tag = [], []
  for w in t:
      tagged_sentence.append((w[1], w[3]))
      sentence.append(w[1])
      bio_tag.append(w[3])

  sentences.append(sentence)
  tags.append(bio_tag)

print("샘플 크기 : \n", len(sentences))
print("0번째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 : \n", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 : \n", (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 61999
0번째 샘플 단어 시퀀스 : 
 ['가락지빵', '주문', '하', '고', '싶', '어요']
0번째 샘플 bio 태그 : 
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O']
샘플 단어 시퀀스 최대 길이 : 
 168
샘플 단어 시퀀스 평균 길이 : 
 8.796238649010467


불러온 말뭉치 데이터에서 단어(w[1])와 BIO태그(w[3])만 불러와 학습용 데이터셋을 생성합니다.

In [ ]:
# 토크나이저 정의
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False: 소문자 변환 X
tag_tokenizer.fit_on_texts(tags)

# 단어 사전 및 태그 사전 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기: ", tag_size)
print("단어 사전 크기: ", vocab_size)

BIO 태그 사전 크기:  10
단어 사전 크기:  17869


단어 시퀀스는 미리 만들어 둔 Preprocess 객체에서 생성하므로 이 예제에서는 BIO 태그용 토크나이저 객체만 생성합니다.

그리고 생성된 사전 리스트를 이용해 단어와 태그 사전의 크기를 정의해줍니다.

In [ ]:
# 학습용 단어 시퀀스 생성
x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
y_train = tag_tokenizer.texts_to_sequences(tags)

index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환
index_to_ner[0] = 'PAD'

입력 문장은 Preprocess에서 생성한 단어 인덱스 시퀀스(get_wordidx_sequence)를 사용하여 번호 형태로 인코딩하고, BIO 태그는 위에서 만든 사전 데이터를 시퀀스 번호 형태로 인코딩합니다.

그리고 모델 학습 후 모델이 예측한 문장의 태그 번호를 다시 NER로 변환해주기 위한 index_to_ner를 만들어줍니다.

In [ ]:
# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

개체명 인식 모델의 입출력 베터 크기를 동일하게 맞추기 위해 단어 시퀀스의 평균 길이보다 넉넉하게 40으로 정의하여 시퀀스 패딩 작업을 실시합니다.

In [ ]:
from sklearn.model_selection import train_test_split
# 학습 데이터와 테스트 데이터 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=1234)

# 출력 데이터 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (49599, 40)
학습 샘플 레이블 형상 :  (49599, 40, 10)
테스트 샘플 시퀀스 형상 :  (12400, 40)
테스트 샘플 레이블 형상 :  (12400, 40, 10)


모델 학습을 위해 학습용과 테스트용 데이터셋을 8:2 비율로 분리합니다.

이후 학습과 테스트용 출력 데이터(y_train, y_test)를 태그 사전 크기에 맞게 원-핫 인코딩을 해줍니다.

In [ ]:
# 모델 정의(Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01),
              metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)

print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
model.save('ner_model.h5')

Epoch 1/10
388/388 [==============================] - 468s 1s/step - loss: 0.1144 - accuracy: 0.9680
Epoch 2/10
388/388 [==============================] - 405s 1s/step - loss: 0.0388 - accuracy: 0.9875
Epoch 3/10
388/388 [==============================] - 406s 1s/step - loss: 0.0255 - accuracy: 0.9917
Epoch 4/10
388/388 [==============================] - 404s 1s/step - loss: 0.0200 - accuracy: 0.9933
Epoch 5/10
388/388 [==============================] - 407s 1s/step - loss: 0.0170 - accuracy: 0.9941
Epoch 6/10
388/388 [==============================] - 405s 1s/step - loss: 0.0150 - accuracy: 0.9949
Epoch 7/10
388/388 [==============================] - 406s 1s/step - loss: 0.0137 - accuracy: 0.9952
Epoch 8/10
388/388 [==============================] - 406s 1s/step - loss: 0.0137 - accuracy: 0.9953
Epoch 9/10
388/388 [==============================] - 404s 1s/step - loss: 0.0125 - accuracy: 0.9956
Epoch 10/10
388/388 [==============================] - 28s 65ms/step - loss: 0.0572 - accur

개체 인식 모델을 순차 모델 방식으로 구현한 코드입니다. tag_size 만큼의 출력 뉴런에서 제일 확률이 높은 출력값 1개를 선택하는 문제이므로 모델 출력 계층의 활성화 함수로 softmax를 사용하였고, 손실 함수는 categorical_crossentropy를 사용했습니다.

마지막으로 학습이 완료된 모델을 이후 챗봇 엔진의 개체명 인식 모듈에서 사용할 수 있도록 h5 파일 포맷으로 저장해줍니다.

In [ ]:
# F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report
import numpy as np

# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences): # 예측값을 index_to_ner을 사용하여 태깅 정보로 변환
    result = []
    for sequence in sequences: # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다
        temp = []
        for pred in sequence: # 시퀀스로부터 예측값을 하나씩 꺼낸다
            pred_index = np.argmax(pred) # ex) [0, 0, 1, 0, 0]이라면 1의 인덱스인 2를 리턴
            temp.append(index_to_ner[pred_index].replace("PAD", "0")) # 'PAD'는 '0'으로 변경
        result.append(temp)

    return result

학습이 완료된 모델을 통해 테스트용 데이터셋의 예측 결과(시퀀스 번호 형태)를 다시 NER 태그로 변환해주기 위한 함수를 만들어줍니다. 해당 함수의 입력은 시퀀스 번호로 인코딩된 테스트용 단어 시퀀스(넘파이 배열)을 사용하고, 해당 함수의 결과로는 예측된 NER 태그 정보가 담긴 넘파일 배열이 반환됩니다.

NER 태그별로 계산된 정밀도(precision)와 재현율(recall), F1 스코어를 출력하는 `seqeval.metrics` 모듈의 classification_report() 함수를 이용합니다. (f1_score() 함수를 통해 F1 스코어값만 불러올 수도 있습니다.)

In [ ]:
# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test) # 실제 NER

# F1 평가 결과
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

388/388 [==============================] - 27s 65ms/step


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: U

              precision    recall  f1-score   support

          NP       1.00      1.00      1.00       303
           _       0.54      0.54      0.54       647
         _DT       1.00      1.00      1.00     13683
       _FOOD       1.00      1.00      1.00     11655
         _LC       0.74      0.61      0.67       314
         _OG       0.50      0.57      0.53       460
         _PS       0.62      0.50      0.55       396
         _TI       0.83      0.72      0.77        61

   micro avg       0.97      0.97      0.97     27519
   macro avg       0.78      0.74      0.76     27519
weighted avg       0.97      0.97      0.97     27519

F1-score: 96.9%


본 모델에서는 예측 결과의 평균 F1 스코어가 96.9%로 나왔고, 이는 이전 3-1편에서 사용한 모델에 비해 더 높은 F1 스코어가 나온 것을 확인할 수 있습니다.

### 4.2 개체명 인식 모듈 생성

이제 앞서 학습한 개체명 인식 모델 파일을 활용해 입력 문장 내부의 개체명을 인식하는 챗봇 엔진의 개체명 인식 모듈을 만들어 볼 차례입니다. 해당 모듈은 딥러닝 모델이기때문에 /models/ner 디렉터리 내에 NerModel.py 소스 파일을 생성하겠습니다.

- **챗봇 엔진 NER 모델 모듈**

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

# 개체명 인식 모델 모듈
class NerModel:
    def __init__(self, model_name, proprocess):

        # BIO 태그 클래스별 레이블
        self.index_to_ner = {1:'0', 2:'B_DT', 3:'B_FOOD', 4:'I', 5:'B_OG',
                             6:'B_PS', 7:'B_LC', 8:'NNP', 9:'B_TI', 0:'PAD'}

        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 Preprocess 객체
        self.p = proprocess

    # 개체명 클래스 예측
    def predict(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장 내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywors(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 패딩처리
        max_len = 40
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding='post', value=0, maxlen=max_len)

        # 키워드별 개체명 예측
        predict = self.model.predict(np.array([padded_seqs[0]]))
        predict_class = tf.math.argmax(predict, axis=-1)

        tags = [self.index_to_ner[i] for i in predict_class.numpy()[0]]
        return list(zip(keywords, tags))

    def predict_tags(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장 내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywors(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 패딩처리
        max_len = 40
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding='post', value=0, maxlen=max_len)

        # 키워드별 개체명 예측
        predict = self.model.predict(np.array([padded_seqs[0]]))
        predict_class = tf.math.argmax(predict, axis=-1)

        tags = []
        for tag_idx in predict_class.numpy()[0]:
            if tag_idx == 1: continue
            tags.append(self.index_to_ner[tag_idx])

        if len(tags) == 0:
            return None
        return tags

앞서 만든 NerModel 클래스를 테스트하는 코드입니다. NerModel 객체를 생성해 새로운 유형의 문장에서 개체명을 인식합니다.

테스트 코드이므로 /test 디렉터리에 model_ner_test.py 파일을 생성해주세요.

- **NerModel 객체 사용**

In [ ]:
from utils.Preprocess import Preprocess
from models.ner.NerModel import NerModel

p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dict.bin',
                  userdic = './utils/user_dic.tsv')

ner = NerModel(model_name='./models/ner/ner_model.h5', proprocess=p)
query = '내일 오후 2시 30분에 짜장면 주문하고 싶어요'
predicts = ner.predict(query)
print(predicts)

1/1 [==============================] - 1s 646ms/step
[('내일', 'B_DT'), ('오후', 'B_DT'), ('2시', 'B_DT'), ('30분', 'B_DT'), ('짜장면', 'B_FOOD'), ('주문', 'O'), ('싶', 'O')]


결과는 다음과 같습니다. 테스트 예문이 학습 데이터 유형과 비슷해 학습한 모델이 개체명들을 잘 인식한 결과를 볼 수 있습니다. 더 다양한 유형의 문장을 학습하면 NER 품질이 더 좋아집니다.

## 5. 답변 검색

챗봇의 답변 검색은 그 자체만으로도 방대한 양의 지식을 필요로 하는 분야이지만 이 게시글에서는 단순한 SQL구문을 사용해 **룰 베이스 기반**으로 답변을 검색하는 방법을 소개하겠습니다.

### 5.1 데이터베이스 제어 모듈 생성

/utils 디렉터리 내에 Database.py 파일을 생성해주세요.

In [ ]:
import pymysql
import pymysql.cursors
import logging


class Database:
    '''
    데이터베이스 제어
    '''

    def __init__(self, host, user, password, db_name, charset='utf8'):
        self.host = host
        self.user = user
        self.password = password
        self.charset = charset
        self.db_name = db_name
        self.conn = None

    # DB 연결
    def connect(self):
        if self.conn is not None:
            return

        self.conn = pymysql.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            db=self.db_name,
            charset=self.charset
        )

    # DB 연결 닫기
    def close(self):
        if self.conn is None:
            return

        if not self.conn.open:
            self.conn = None
            return
        self.conn.close()
        self.conn = None

    # SQL 구문 실행
    def execute(self, sql):
        last_row_id = -1
        try:
            with self.conn.cursor() as cursor:
                cursor.execute(sql)
            self.conn.commit()
            last_row_id = cursor.lastrowid
            # logging.debug("execute last_row_id : %d", last_row_id)
        except Exception as ex:
            logging.error(ex)

        finally:
            return last_row_id

    # SELECT 구문 실행 후 단 1개의 데이터 ROW만 불러옴
    def select_one(self, sql):
        result = None

        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
                cursor.execute(sql)
                result = cursor.fetchone()
        except Exception as ex:
            logging.error(ex)

        finally:
            return result

    # SELECT 구문 실행 후 전체 데이터 ROW를 불러옴
    def select_all(self, sql):
        result = None

        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
                cursor.execute(sql)
                result = cursor.fetchall()
        except Exception as ex:
            logging.error(ex)

        finally:
            return result

### 5.2 답변 검색 모듈 생성

챗봇/utils 디렉터리에 FindAnswer.py 파일에 코드를 작성해주세요.

In [ ]:
class FindAnswer:
    def __init__(self, db):
        self.db = db

    # 검색 쿼리 생성
    def _make_query(self, intent_name, ner_tags):
        sql = "select * from chatbot_train_data"
        if intent_name is not None and ner_tags is None:
            sql = sql + " where intent='{}' ".format(intent_name)

        elif intent_name is not None and ner_tags is not None:
            where = " where intent='%s' " % intent_name
            if len(ner_tags) > 0:
                where += "and ("
                for ne in ner_tags:
                    where += " ner like '%{}%' or ".format(ne)
                where = where[:-3] + ')'
            sql = sql + where

        # 동일한 답변이 2개 이상인 경우 랜덤으로 선택
        sql = sql + " order by rand() limit 1"
        return sql

    # 답변 검색
    def search(self, intent_name, ner_tags):
        # 의도명과 개체명으로 답변 검색
        sql = self._make_query(intent_name, ner_tags)
        answer = self.db.select_one(sql)

        # 검색되는 답변이 없으면 의도명만 검색
        if answer is None:
            sql = self._make_query(intent_name, None)
            answer = self.db.select_one(sql)

        return answer['answer'], answer['answer_image']

    # NER 태그를 실제 입력된 단어로 변환
    def tag_to_word(self, ner_predicts, answer):
        for word, tag in ner_predicts:

            # 변환해야 하는 태그가 있는 경우 추가
            if tag == 'B_FOOD':
                answer = answer.replace(tag, word)

        answer = answer.replace('{', '')
        answer = answer.replace('}', '')
        return answer

## 6. 챗봇 엔진 서버 개발

다양한 플랫폼에서 언제든지 사용할 수 있도록 서버용 프로그램을 만들어야합니다. 이 절에서는 서버 통신 기능을 구현하겠습니다.

### 6.1 통신 프로토콜 정의

챗봇 엔진 서버와 통신하기 위해서는 프로토콜이 필요합니다. 프로토콜이란 서버와 클라이언트 간의 통신을 위한 규약입니다. Key/Value 쌍으로 이루어진 데이터 객체를 전달하는 JSON 형태를 주로 사용합니다. 챗봇 엔진은 양방향 통신이기 때문에 두 가지 형태의 프로토콜을 정의해야합니다.

먼저, 클라이언트에서 서버 쪽으로 요청하는 JSON 프로토콜 예시입니다.

Query는 챗봇 엔진에 요청하는 질의 텍스트이고 BotType은 서버에 접속하는 챗봇 서비스 타입입니다.

```json
{
	"Query" : "자장면 주문할게요",
	"BotType" : "Kakao"
}
```

다음은 챗봇 엔진의 처리 결과를 클라이언트 쪽에 응답하는 JSON 프로토콜 예시입니다.

Query는 챗봇 엔진에 요청한 질의 텍스트이며, Intent는 챗봇 엔진이 해석한 질의 텍스트의 의도, NER은 인ㄴ식된 개체명, Answer는 요청한 질의의 답변 텍스트입니다. 답변에 이미지가 존재하는 경우 AnswerInmageUrl에 이미지 경로가 있습니다.

```json
{
	"Query" : "자장면 주문할게요",
	"Intent" : "주문",
	"NER" : "[('자장면', 'B_FOOD'), ('주문', 'O')]",
	"Answer" : "자장면 주문 처리 감사",
	"AnswerImageUrl" : ""
}
```

아래 그림은 서버와 클라이언트 간에 JSON 데이터를 주고받는 과정입니다.

### 6.2 다중 접속을 위한 TCP 소켓 서버

지금가지는 한 번에 하나의 작업만 실행하는 **싱글 스레드(single thread)** 방식이었습니다. 다른 클라이언트가 서비스를 받고 있는 경우에는 챗봇 엔진의 응답을 받지 못하는 문제가 있습니다.

그래서 **멀티 스레드(multi thread)** 방식을 사용합니다. **멀티 스레드**란 하나의 프로그램이 동시에 여러 개의 작업을 할 수 있도록 하는 방법입니다. 여기서 스레드란 프로그램 내에서 실행되는 단위입니다. 어떤 함수를 하나 호출하면 하나의 스레드가 생겨서 실행됩니다.

챗봇 엔진 서버는 챗봇 클라이언트가 연결 요청을 할 때마다 챗봇 엔진 처리 스레드를 생성해 일련의 과정을 거처 요청한 질의의 답변을 클라이언트 쪽으로 전송합니다. 챗봇 클라이언트의 요청이 동시에 들어온다면 챗봇 엔진 처리 스레드는 클라이언트 연결 요청 수만큼 생성되어 동작합니다.

TCP 소켓 서버를 관리하는 모듈을 먼저 만들어봅시다. 이 모듈은 서버에 접속하는 클라이언트 소켓을 생성하고 처리하는 기능을 합니다. /utils 디렉터리 내에 [BotServer.py](http://BotServer.py) 파일을 생성해주세요

In [ ]:
import socket

class BotServer:
    def __init__(self, srv_port, listen_num):
        self.port = srv_port
        self.listen = listen_num
        self.mySock = None

    # sock 생성
    def create_sock(self):
        self.mySock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.mySock.bind(("0.0.0.0", int(self.port)))
        self.mySock.listen(int(self.listen))
        return self.mySock

    # client 대기
    def ready_for_client(self):
        return self.mySock.accept()

    # sock 반환
    def get_sock(self):
        return self.mySo

다음은 챗봇 엔진 서버 프로그램을 완성하겠습니다. BotServer 클래스와 멀티 스레드 모듈을 이용합니다. 다음 코드는 챗봇 엔진 메인 프로그램이기 때문에 프로젝트 루트 디렉터리에 bot.py 파일로 생성합니다.

```python
import threading
import json

from config.DatabaseConfig import *
from utils.Database import Database
from utils.BotServer import BotServer
from utils.Preprocess import Preprocess
from models.intent.IntentModel import IntentModel
from models.ner.NerModel import NerModel
from utils.FindAnswer import FindAnswer


# 전처리 객체 생성
p = Preprocess(word2index_dic='train_tools/dict/chatbot_dict.bin',
               userdic='utils/user_dic.tsv')

# 의도 파악 모델
intent = IntentModel(model_name='models/intent/intent_model.h5', proprocess=p)

# 개체명 인식 모델
ner = NerModel(model_name='models/ner/ner_model.h5', proprocess=p)


def to_client(conn, addr, params):
    db = params['db']

    try:
        db.connect()  # 디비 연결

        # 데이터 수신
        read = conn.recv(2048)  # 수신 데이터가 있을 때 까지 블로킹
        print('===========================')
        print('Connection from: %s' % str(addr))

        if read is None or not read:
            # 클라이언트 연결이 끊어지거나, 오류가 있는 경우
            print('클라이언트 연결 끊어짐')
            exit(0)


        # json 데이터로 변환
        recv_json_data = json.loads(read.decode())
        print("데이터 수신 : ", recv_json_data)
        query = recv_json_data['Query']

        # 의도 파악
        intent_predict = intent.predict_class(query)
        intent_name = intent.labels[intent_predict]

        # 개체명 파악
        ner_predicts = ner.predict(query)
        ner_tags = ner.predict_tags(query)


        # 답변 검색
        try:
            f = FindAnswer(db)
            answer_text, answer_image = f.search(intent_name, ner_tags)
            answer = f.tag_to_word(ner_predicts, answer_text)

        except:
            answer = "죄송해요 무슨 말인지 모르겠어요. 조금 더 공부 할게요."
            answer_image = None

        send_json_data_str = {
            "Query" : query,
            "Answer": answer,
            "AnswerImageUrl" : answer_image,
            "Intent": intent_name,
            "NER": str(ner_predicts)
        }
        message = json.dumps(send_json_data_str)
        conn.send(message.encode())

    except Exception as ex:
        print(ex)

    finally:
        if db is not None: # db 연결 끊기
            db.close()
        conn.close()


if __name__ == '__main__':

    # 질문/답변 학습 디비 연결 객체 생성
    db = Database(
        host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME
    )
    print("DB 접속")

    port = 5050
    listen = 100

    # 봇 서버 동작
    bot = BotServer(port, listen)
    bot.create_sock()
    print("bot start")

    while True:
        conn, addr = bot.ready_for_client()
        params = {
            "db": db
        }

        client = threading.Thread(target=to_client, args=(
            conn,
            addr,
            params
        ))
        client.start()
```

드디어 챗봇 엔진 서버가 완성되었습니다. 해당 프로그램이 서버 환경에서 백그라운드 프로그램으로 동작한다면 24시간 챗봇 서비스를 운영할 수 있습니다.

마지막으로 챗봇 엔진 서버 프로그램에 접속하는 클라이언트 프로그램을 만들어서 챗봇 엔진 동작을 테스트해봅시다.

### 6.3 챗봇 테스트 클라이언트 프로그램

챗봇 엔진 서버를 테스트할 수 있는 클라이언트 프로그램을 간단하게 만들어보겠습니다. 챗봇 테스트 클라이언트 프로그램은 콘솔 화면에 질문을 입력하면 챗봇 엔진 서버와 통신해 결과를 출력합니다. /test 디렉터리에 chatbot_client_test.py 파일을 생성합니다.

```python
import socket
import json

# 챗봇 엔진 서버 접속 정보
host = "127.0.0.1"  # 챗봇 엔진 서버 IP 주소
port = 5050  # 챗봇 엔진 서버 통신 포트

# 클라이언트 프로그램 시작
while True:
    print("질문 : ")
    query = input()  # 질문 입력
    if(query == "exit"):
        exit(0)
    print("-" * 40)

    # 챗봇 엔진 서버 연결
    mySocket = socket.socket()
    mySocket.connect((host, port))

    # 챗봇 엔진 질의 요청
    json_data = {
        'Query': query,
        'BotType': "MyService"
    }
    message = json.dumps(json_data)
    mySocket.send(message.encode())

    # 챗봇 엔진 답변 출력
    data = mySocket.recv(2048).decode()
    ret_data = json.loads(data)
    print("답변 : ")
    print(ret_data['Answer'])
    print(ret_data)
    print(type(ret_data))
    print("\n")

    # 챗봇 엔진 서버 연결 소켓 닫기
    mySocket.close()
```

우리가 학습한 데이터를 토대로 적절한 답변을 출력하고 있습니다. 더 많은 데이터로 다양한 의도와 개체명, 질문에 맞는 답변을 보강한다면 멋진 챗봇을 만들 수 있을것입니다!